> Updated in PyMongo 3.11.0

In [ ]:
from datetime import datetime

In [ ]:
import pymongo
from pymongo import MongoClient

### Establishing Connection

#### Documentation
- [Connection String](https://docs.mongodb.com/manual/reference/connection-string)
- [MongoClient](https://pymongo.readthedocs.io/en/stable/api/pymongo/mongo_client.html)

In [ ]:
# connecting to localhost, listening on port 27017
client = MongoClient('mongodb://localhost:27017')

### Check Database Names

In [ ]:
# get a list of the names of all databases on the connected server
client.list_database_names()

### Create/Access Database and Collection

In [ ]:
""" Accessing a database/collection will create one
if it does not already exist.
"""

# get the db_name Database on MongoClient client 
db = client.db_name
# get the collection_name Collection of Database db
collection = db.collection_name

### Importing Data with mongoimport

`mongoimport` a Unix command which comes with Mongo for importing data

`%%bash` Jupyter magic for bash command in the next cell 

`%lsmagic` List out available Jupyter \"magics\"

#### Documentation
- [mongoimport](https://docs.mongodb.com/database-tools/mongoimport/)

In [1]:
%%bash
# wget to download the JSON file from the url
wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
# specifies the database, collection, and import file
# --drop drop any collection with the same name which already exists
mongoimport --db test --collection restaurants --drop --file ./primer-dataset.json
# delete the JSON file
rm ./primer-dataset.json

--2020-11-04 15:34:56--  https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11874761 (11M) [text/plain]
Saving to: â€˜primer-dataset.jsonâ€™

     0K .......... .......... .......... .......... ..........  0%  857K 13s
    50K .......... .......... .......... .......... ..........  0%  623K 16s
   100K .......... .......... .......... .......... ..........  1%  721K 16s
   150K .......... .......... .......... .......... ..........  1%  297K 21s
   200K .......... .......... .......... .......... ..........  2%  403K 23s
   250K .......... .......... .......... .......... ..........  2%  395K 24s
   300K .......... .......... .......... .......... ..........  3% 30.1K 74s
   350K .......... .......... .......... .

### Inserting Documents

In [ ]:
""" The way to insert data is as a JSON object. 
For Python, use a `dict` or a `list` for this.

PyMongo automatically adds a unique identifier `_id` field to a document
if it does not already exist.
"""

# create an object and insert into the collection
single_obj = {'name': 'Amber',
              'Meaning of life': 42, 
              'time': datetime.now()}
collection.insert_one(single_obj)

# create a list of object and insert into the collection
many_objects = ({'name': 'RAY'}, {'name': 'MATTHEW'})
collection.insert_many(many_objects)

## Querying Documents

### [Find](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find)
`find()` returns a [cursor](https://api.mongodb.com/python/current/api/pymongo/cursor.html) of all documents in the collection
matching the query condition.

The `filter` parameter is a dict, 
which searches for the documents where `key` = `value` 
where the dict is of the form `{key: value}`

The `limit` parameter specifies the maximum number of results to return

In [5]:
# returns a single document matching the query condition
collection.find_one() 
collection.find_one({'name':'Robert'})

{u'_id': ObjectId('5dd7c4135ccae9021c6c8d07'),
 u'favourite_song': u'The Load-Out',
 u'name': u'Amber',
 u'star_sign': u'Capricorn'}

In [ ]:
# get all documents in the collection
cursor = collection.find()

# get documents matching the filter condition
cursor = collection.find({'cuisine': 'Bakery'}).limit(5)
cursor = collection.find({'cuisine': 'Bakery'}, limit=5)

# filter with AND conditions
cursor = collection.find({'cuisine': 'Hamburgers', 'borough': 'Manhattan'})

# iterate the cursor to print the results
for doc in cursor:
    print(doc)

### Filters with [Operators](https://docs.mongodb.com/manual/reference/operator/query/)

MongoDB has a series of operators which allow us to do more sophisticated filters on our queries. The specific syntax varies depending on the operator, so it isn't possible to give a general rule.

In [ ]:
# performs a logical OR operation
filter = {"$or": [{"cuisine": "Polynesian"}, {"cuisine": "Hawaiian"}]}

# searches for a regular expression on a particular field
filter = {"name": {"$regex": '^Pretzel'}}
filter = {'borough': 'Brooklyn', 'name': {'$regex': 'Bar$'}}

# comparison between two values where one is greater than the other
filter = {'grades.score': {'$gt': 12}}

# the value of a field equals the specified value
day = datetime(2012, 12, 15)
filter = {'grades.date': {'$eq': day}}

### Organising output with Fields and Sort

The field selection or `projection` argument is the argument after the \[optional\] filter, and is either:
- A list of fields to include (plus \_id)
- A dict of fields with True/False to include

In [35]:
fields = {'_id': False, 'name': True}
collection.find_one(filter, fields)

{u'name': u'Churrascaria Plataforma'}

The `sort` argument is a list of (key, direction) pairs specifying the sort order for this query. This can also be a function in its own right [`sort()`](https://api.mongodb.com/python/current/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort)

In [ ]:
# The ASCENDING constant = 1
# The DESCENDING constant = -1

# sort in alphabetical order
sort = [('name', pymongo.ASCENDING)]
cursor = collection.find(filter, projection=fields, sort=sort)

### [Count](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.count_documents)
`count()` is deprecated in the MongoDB drivers compatible with the 4.0 features 

Check [`db.collection.count()`](https://docs.mongodb.com/manual/reference/method/db.collection.count/) and [Changelog](https://pymongo.readthedocs.io/en/stable/changelog.html) for more info

In [ ]:
# count documents matching the filter condition
count = collection.count_documents({'cuisine': 'Bakery'})

### [Distinct](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.distinct)

In [ ]:
# list out distinct values for key among all documents in this collection
collection.distinct('cuisine')

### Sub-documents

A valid JSON style "document" can have another JSON document inside it.  To access these, we use the "dot" notation to access them.

## MongoDB Aggregation Framework

The framework works as a pipeline, with a series of different stages where the data are transformed in each one.

### Aggregation Example

In [ ]:
""" obtain count, min, max in the collection

Note that `$_id: None` key/value pair in it.  
It is compulsory for a `$group` pipeline to have one, 
and it indicates what it is grouping by. 
In this case, we haven't grouped it at all, 
"""

group = {
    '$group': {
        '_id': None, 
        'size': {'$sum': 1},
        'min': {'$min': '$restaurant_id'},
        'max': {'$max': '$restaurant_id'}
    }
}

cursor = collection.aggregate([group])

In [52]:
""" find the number of different cuisines in Bronx
sort by descending order of counts, otherwise alphabetical order
limit the results to only cuisines with a count of 20 or more
"""

# Restrict the results to only establishments in the Bronx.  
match = {
    "$match": {"borough": "Bronx"}
}

# _id is the field to perform the operation on (like SQL GROUP BY)
# count is the name of the field that the result will be in
# $sum is the counting operation, and the value 1 is how many to count each time
group = {
    '$group': {'_id': '$cuisine', 'count': {'$sum': 1}}
}

# project the results with the Fields 'cuisine' and 'count'
# change the _id field to cuisine
project = {
    '$project': {'_id': False, 'cuisine': '$_id', 'count': True}
}

# count is the field to sort by, and -1 means to sort in descending order
# sorted by alphabetical order for same count
sort = {
    '$sort': {'count': pymongo.DESCENDING, 'Cuisine': pymongo.ASCENDING}
}

# limit the results to only result with a count of > 20
# $match operation should be before $sort and $project to improve speed
limit = {
    "$match": {"count": {'$gt': 20}}
}

pipeline = [match, group, project, sort, limit]
cursor = collection.aggregate(pipeline)
for c in cursor:
    print(c)

{u'Cuisine': u'American', u'count': 411}
{u'Cuisine': u'Chinese', u'count': 323}
{u'Cuisine': u'Pizza', u'count': 197}
{u'Cuisine': u'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 u'count': 187}
{u'Cuisine': u'Spanish', u'count': 127}
{u'Cuisine': u'Caribbean', u'count': 110}
{u'Cuisine': u'Chicken', u'count': 108}
{u'Cuisine': u'Mexican', u'count': 89}
{u'Cuisine': u'Other', u'count': 86}
{u'Cuisine': u'Hamburgers', u'count': 78}
{u'Cuisine': u'Bakery', u'count': 71}
{u'Cuisine': u'Donuts', u'count': 68}
{u'Cuisine': u'Pizza/Italian', u'count': 53}
{u'Cuisine': u'Italian', u'count': 52}
{u'Cuisine': u'Sandwiches', u'count': 49}
{u'Cuisine': u'Caf\xe9/Coffee/Tea', u'count': 45}
{u'Cuisine': u'Juice, Smoothies, Fruit Salads', u'count': 35}
{u'Cuisine': u'African', u'count': 31}
{u'Cuisine': u'Ice Cream, Gelato, Yogurt, Ices', u'count': 27}
{u'Cuisine': u'Delicatessen', u'count': 26}
{u'Cuisine': u'Seafood', u'count': 26}
